In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
data = pd.read_csv("https://github.com/Krow10/code4rena-scraper/raw/master/leaderboard_code4rena.csv") # Set path accordingly

In [3]:
data

,period,handle,is_team,prize_money,total_reports,high_all,high_solo,med_all,med_solo,gas_all
0,2022,WatchPug,True,419294.49,450,89,27,131,46,156
1,2022,cmichel,False,402556.26,148,32,5,67,23,4
2,2022,Spearbit,True,292640.04,5,2,0,1,0,1
3,2022,Saw-mon_and_Natalie,False,246448.61,5,2,0,0,0,1
4,2022,hyh,False,241116.82,278,47,2,94,25,29
...,...,...,...,...,...,...,...,...,...,...
2181,ALL TIME,lyncurion,False,0.00,2,0,0,0,0,1
2182,ALL TIME,0xzh,False,0.00,2,0,0,0,0,1
2183,ALL TIME,alexxander,False,0.00,1,0,0,1,0,0
2184,ALL TIME,navinavu,False,0.00,1,0,0,1,0,0


In [4]:
plt_data = data.copy()

### Period dropdown
Filter from which period the data should be taken. These are the same options as the [leaderboard](https://code4rena.com/leaderboard/) entries from the Code4rena website.

In [5]:
period_dropdown = alt.binding_select(options=list(plt_data["period"].unique()), name='Select a period:')
period_selector = alt.selection_single(
    fields=['period'], 
    bind=period_dropdown, 
    name="period_selector", 
    init={'period':'ALL TIME'}
)

In [6]:
is_team_legend=['Solo', 'Team']
is_team_colors = ['#2c385c', '#ffb412']
plt_data["is_team"] = plt_data["is_team"].map(lambda x: is_team_legend[1 if x else 0])

for col in plt_data.columns[3:]:
    plt_data['sum_' + col] = plt_data.groupby(['period', 'is_team'])[col].transform('sum')

### Total prize money pie chart

In [7]:
total_prize_base = alt.Chart(plt_data).transform_joinaggregate(
    total_prize_money='sum(prize_money)'
).transform_calculate(
    prize_share='datum.sum_prize_money / datum.total_prize_money'
).encode(
    theta=alt.Theta("sum(prize_money):Q", stack=True),
    color=alt.Color(
        "is_team:N",
        scale=alt.Scale(range=is_team_colors),
        legend=alt.Legend(
            title='Wardens',
            orient='top',
            labelFontSize=16, 
            titleFontSize=16,
            symbolType='square',
        )
    ),
    tooltip=['is_team:N', alt.Tooltip('sum_prize_money:Q', format='$,'), alt.Tooltip('prize_share:Q', format='.2%')]
)

In [8]:
pie_radius = 150

In [9]:
total_prize_pie = total_prize_base.mark_arc(
    outerRadius=pie_radius
)

In [10]:
total_prize_labels = total_prize_base.mark_text(
    radius=pie_radius*1.15,
    size=pie_radius/10
).encode(
    text=alt.Text("prize_share:N", format='.2%')
)

In [11]:
total_pie_chart = (total_prize_pie + total_prize_labels).properties(
    title="Total prize money ($USD)",
    width=2.5*pie_radius,
    height=2.5*pie_radius
).add_selection(
    period_selector
).transform_filter(
    period_selector
)

### Repeated pie charts for other data

In [12]:
repeat_pie_titles = dict(zip(
    plt_data.columns[4:10], 
    ['Totals reports', 'High severity', 'Unique high', 'Medium severity', 'Unique medium', 'Gas optimizations']
))

In [13]:
repeat_pie_charts = alt.hconcat()
for col in plt_data.columns[4:10]:
    repeat_pie_charts |= \
        alt.Chart(plt_data).transform_joinaggregate(
            total='sum(' + col + ')', groupby=['period']
        ).transform_calculate(
            share='datum.sum_' + col + ' / datum.total'
        ).mark_arc(
            outerRadius=pie_radius/2
        ).encode(
            theta=alt.Theta(
                col, 
                type='quantitative', 
                aggregate='sum', 
                stack=True
            ),
            color=alt.Color(
                'is_team:N', 
                scale=alt.Scale(range=is_team_colors), 
                legend=None
            ),
            tooltip=[
                'is_team:N', 
                alt.Tooltip(col, type='quantitative', aggregate='sum'),
                alt.Tooltip('share:Q', format='.2%')
            ]
        ).properties(
            title=repeat_pie_titles[col],
            #fontSize=14,
            width=pie_radius,
            height=pie_radius
        ).add_selection(
            period_selector
        ).transform_filter(
            period_selector
        )
repeat_pie_charts = repeat_pie_charts.resolve_scale(theta='independent')

### Grouped bar chart Y-scale
Set this variable to either ``'log'`` for using a logarithmic scale (easier to make comparisons), or ``'linear'`` (easier to see differences of values).

In [14]:
grouped_bar_chart_y_scale = 'linear' # 'linear' | 'log'

In [15]:
plt_bar_data = plt_data.copy()
plt_bar_data.columns = plt_bar_data.columns.tolist()[:11] + list(repeat_pie_titles.values())
columns_order = dict(zip(list(repeat_pie_titles.values()), list(range(len(repeat_pie_titles)))))

In [16]:
grouped_bar_chart = alt.Chart(plt_bar_data).transform_fold(
    [col for col in list(repeat_pie_titles.values())]
).transform_calculate(
    order=str(columns_order)+'[datum.key]'
).mark_bar().encode(
    x=alt.X('is_team:O', title=None, axis=None),
    y=alt.Y('value:Q', scale=alt.Scale(type=grouped_bar_chart_y_scale), axis=alt.Axis(title=None)),
    color='is_team:N',
    column=alt.Column('key:N', title=None, sort=alt.SortField("order", order="ascending")),
    tooltip=['is_team:N', alt.Tooltip('value:Q')],
).add_selection(
    period_selector
).transform_filter(
    period_selector
).properties(
    width=alt.Step(pie_radius/2),
    height=200
)

# Comparison of solo wardens VS teams per period 

In [17]:
alt.vconcat(total_pie_chart, repeat_pie_charts, grouped_bar_chart, center=True).configure_title(
    fontSize=22
).configure_view(
    strokeWidth=0
)

/opt/hostedtoolcache/Python/3.11.1/x64/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)